# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [4]:
# import libraries
import sqlite3
import pandas as pd
import re
import nltk
from nltk.tokenize import word_tokenize
nltk.download(['punkt', 'wordnet', 'stopwords'])
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.ensemble  import RandomForestClassifier
from sklearn.multioutput import MultiOutputClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import GridSearchCV
from sklearn.naive_bayes import MultinomialNB


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\puet_fl\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\puet_fl\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\puet_fl\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
# load data from database
#conn = sqlite3.connect("DisasterResponse.db")
#df = pd.read_sql("SELECT * FROM DisasterResponse", conn)
#X = 
#Y = 

In [6]:
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('sqlite:///DisasterResponse.db')
df = pd.read_sql("DisasterResponse", engine)
X = df["message"]
y = df.iloc[:, 4:]
df.head(1)


,id,message,original,genre,related,request,offer,aid_related,medical_help,medical_products,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,unrelated
0,2,Weather update - a cold front from Cuba that c...,Un front froid se retrouve sur Cuba ce matin. ...,direct,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
X.head(1)


0    Weather update - a cold front from Cuba that c...
Name: message, dtype: object

In [8]:
y.head(1)

,related,request,offer,aid_related,medical_help,medical_products,search_and_rescue,security,military,child_alone,...,other_infrastructure,weather_related,floods,storm,fire,earthquake,cold,other_weather,direct_report,unrelated
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### 2. Write a tokenization function to process your text data

In [9]:
df.shape

(26028, 41)

In [10]:
def tokenize(text, use_stemming=True):
    text = re.sub(r"[^\w\s]"," ", text.lower())
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()

    clean_tokens = []
    for tok in tokens:
        if tok in stopwords.words("english"):
            continue
        clean_tok = lemmatizer.lemmatize(tok).strip()
        clean_tokens.append(clean_tok)

    if use_stemming:
        clean_tokens = [PorterStemmer().stem(w) for w in clean_tokens]
    return clean_tokens

tokenize("This is just a Test of the 3 tokenize go gone going went ongoing function!", True)
    

['test', '3', 'token', 'go', 'gone', 'go', 'went', 'ongo', 'function']

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [11]:
classifier = RandomForestClassifier()
pipeline = Pipeline([
    ("vect", CountVectorizer(tokenizer = tokenize)),
    ("tfidf", TfidfTransformer()),
    ("model", MultiOutputClassifier(classifier))
])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001952983EC10>)),
                ('tfidf', TfidfTransformer()),
                ('model',
                 MultiOutputClassifier(estimator=RandomForestClassifier()))])

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [13]:
def print_scores(y_test, y_pred):
    # Printing the classification report for each label
    for count, column in enumerate(y_test.columns):
        print("Label:", column)
        print(classification_report(y_test[column], y_pred[:, count]))
              
    accuracy = (y_pred == y_test.values).mean()
    print(f"model accuracy {accuracy}")
    
y_pred = pipeline.predict(X_test)
print_scores(y_test, y_pred)

Label: related
              precision    recall  f1-score   support

           0       0.73      0.42      0.53      1580
           1       0.84      0.95      0.89      4927

    accuracy                           0.82      6507
   macro avg       0.78      0.69      0.71      6507
weighted avg       0.81      0.82      0.80      6507

Label: request
              precision    recall  f1-score   support

           0       0.90      0.98      0.94      5383
           1       0.83      0.47      0.60      1124

    accuracy                           0.89      6507
   macro avg       0.87      0.73      0.77      6507
weighted avg       0.89      0.89      0.88      6507

Label: offer
              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6473
           1       0.00      0.00      0.00        34

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      0.

C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

              precision    recall  f1-score   support

           0       0.96      1.00      0.98      6216
           1       0.88      0.13      0.23       291

    accuracy                           0.96      6507
   macro avg       0.92      0.56      0.60      6507
weighted avg       0.96      0.96      0.95      6507

Label: other_aid
              precision    recall  f1-score   support

           0       0.87      1.00      0.93      5658
           1       0.61      0.03      0.06       849

    accuracy                           0.87      6507
   macro avg       0.74      0.51      0.49      6507
weighted avg       0.84      0.87      0.82      6507

Label: infrastructure_related
              precision    recall  f1-score   support

           0       0.93      1.00      0.97      6084
           1       0.00      0.00      0.00       423

    accuracy                           0.93      6507
   macro avg       0.47      0.50      0.48      6507
weighted avg       0.87    

C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

In [14]:
y_pred == y_test.values

array([[ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       ...,
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True],
       [ True,  True,  True, ...,  True,  True,  True]])

In [15]:
print(y_pred.shape)
print(y_test.shape)

(6507, 37)
(6507, 37)


In [16]:
y_test.iloc[:,0]

16079    1
23736    1
15531    1
14035    1
10949    1
        ..
14657    1
21943    1
22863    0
18741    1
17121    1
Name: related, Length: 6507, dtype: int64

### 6. Improve your model
Use grid search to find better parameters. 

In [17]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x000001952983EC10>)),
  ('tfidf', TfidfTransformer()),
  ('model', MultiOutputClassifier(estimator=RandomForestClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x000001952983EC10>),
 'tfidf': TfidfTransformer(),
 'model': MultiOutputClassifier(estimator=RandomForestClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text, use_stemming=True)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf_

In [18]:
parameters = {
    'tfidf__use_idf': [True, False],
    'model__estimator__n_estimators': [10, 100],
    #'model__estimator__min_samples_split': (2, 4),
    'model__estimator__bootstrap': [True, False],
}

cv = GridSearchCV(pipeline, parameters)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [19]:
cv.fit(X_train, y_train)

GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x000001952983EC10>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('model',
                                        MultiOutputClassifier(estimator=RandomForestClassifier()))]),
             param_grid={'model__estimator__bootstrap': [True, False],
                         'model__estimator__n_estimators': [10, 100],
                         'tfidf__use_idf': [True, False]})

In [20]:
cv.best_params_

{'model__estimator__bootstrap': False,
 'model__estimator__n_estimators': 100,
 'tfidf__use_idf': True}

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [21]:
classifier = MultinomialNB()
pipeline_nb = Pipeline([
    ("vect", CountVectorizer(tokenizer = tokenize)),
    ("tfidf", TfidfTransformer()),
    ("model", MultiOutputClassifier(classifier))
])

In [22]:
X_train, X_test, y_train, y_test = train_test_split(X, y)
pipeline_nb.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x000001952983EC10>)),
                ('tfidf', TfidfTransformer()),
                ('model', MultiOutputClassifier(estimator=MultinomialNB()))])

In [23]:
y_pred = pipeline_nb.predict(X_test)
print_scores(y_test, y_pred)

Label: related
              precision    recall  f1-score   support

           0       0.80      0.11      0.19      1485
           1       0.79      0.99      0.88      5022

    accuracy                           0.79      6507
   macro avg       0.80      0.55      0.53      6507
weighted avg       0.79      0.79      0.72      6507

Label: request
              precision    recall  f1-score   support

           0       0.86      0.99      0.92      5371
           1       0.86      0.25      0.39      1136

    accuracy                           0.86      6507
   macro avg       0.86      0.62      0.66      6507
weighted avg       0.86      0.86      0.83      6507

Label: offer
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6485
           1       0.00      0.00      0.00        22

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.

C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

              precision    recall  f1-score   support

           0       0.90      1.00      0.95      5886
           1       0.00      0.00      0.00       621

    accuracy                           0.90      6507
   macro avg       0.45      0.50      0.47      6507
weighted avg       0.82      0.90      0.86      6507

Label: clothing
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6397
           1       0.00      0.00      0.00       110

    accuracy                           0.98      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.97      0.98      0.97      6507

Label: money
              precision    recall  f1-score   support

           0       0.98      1.00      0.99      6357
           1       0.00      0.00      0.00       150

    accuracy                           0.98      6507
   macro avg       0.49      0.50      0.49      6507
weighted avg       0.95      0.98      0.97  

C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

              precision    recall  f1-score   support

           0       0.99      1.00      1.00      6470
           1       0.00      0.00      0.00        37

    accuracy                           0.99      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      0.99      0.99      6507

Label: hospitals
              precision    recall  f1-score   support

           0       0.99      1.00      0.99      6437
           1       0.00      0.00      0.00        70

    accuracy                           0.99      6507
   macro avg       0.49      0.50      0.50      6507
weighted avg       0.98      0.99      0.98      6507

Label: shops
              precision    recall  f1-score   support

           0       1.00      1.00      1.00      6475
           1       0.00      0.00      0.00        32

    accuracy                           1.00      6507
   macro avg       0.50      0.50      0.50      6507
weighted avg       0.99      1.00      0.99 

C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\puet_fl\AppData\Local\miniforge3\lib\site-packages\sklearn\metrics\_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\U

              precision    recall  f1-score   support

           0       0.83      0.99      0.90      5214
           1       0.79      0.18      0.29      1293

    accuracy                           0.83      6507
   macro avg       0.81      0.58      0.59      6507
weighted avg       0.82      0.83      0.78      6507

Label: unrelated
              precision    recall  f1-score   support

           0       0.79      0.99      0.88      5022
           1       0.80      0.11      0.19      1485

    accuracy                           0.79      6507
   macro avg       0.80      0.55      0.53      6507
weighted avg       0.79      0.79      0.72      6507

model accuracy 0.9327086422522107
